In [161]:
# Analysis of chip data; prepare in separate pipelines
import pandas as pd
import numpy as np
import os, sys
import matplotlib as plt
import seaborn as sns
# Data load
filename = 'chip_dataset.csv'
file = os.path.abspath(os.path.join(os.getcwd(),'..', filename))
df = pd.read_csv(file)
# df.head()

In [162]:
target_label = ['Type',  'Foundry']
df.drop(['Unnamed: 0', 'Product', 'Release Date'], axis = 1, inplace=True)

In [163]:
# Util to get NaNs
def get_null_pc(df1, out = False) :
    gt_threshold = 0.58
    null_map = {'gt':{}, 'lt':{}, 'none' : {}, 'other':{}}
    null_pc_map = 1 - df1.count()/len(df1)
    if out == True :
        print(null_pc_map)
    for key, val in null_pc_map.items() :
        pc_type = 'other'
        if val == 0.0 :
            pc_type = 'none'
        elif val > gt_threshold :
            pc_type = 'gt'
        elif val < 0.05 and val != 0 :
            pc_type = 'lt'

        if key not in null_map[pc_type] :
            null_map[pc_type][key] = [val]
        else :
            null_map[pc_type][key].append(val)
        
    return null_map
# print(df.info())
null_pc = get_null_pc(df, out = False)
# print(df.shape)

In [164]:
# First round of null drops
# Drop rows with < 5% is NaNs
# print ("Dropping rows of ", list(null_pc['lt'].keys()))
df.dropna(subset = null_pc['lt'].keys(), inplace=True)

# Drop entire col if col > 85% NaNs
# print ("Dropping cols : ", null_pc['gt'].keys())
if len(null_pc['gt'].keys()) > 0 :
    df.drop(null_pc['gt'].keys(), axis = 1, inplace = True)
# print (df.info())

In [165]:
# Split X + Y
X = df.loc[:, list(set(df.columns) - set(target_label))]
Y = df.loc[:, target_label]

In [166]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
# Transform target label
les = []
for label in target_label :
    le = LabelEncoder()
    Y[label] =  le.fit_transform(Y[label])
    les.append(le)

# One hot encode rest of categorical data 
# #TBD: Study ordinal vs binary effect on scores
enc = OneHotEncoder(sparse_output=False)
obj_vals = list(set(X.columns) - set(X._get_numeric_data().columns))
enc.fit(X.loc[:,obj_vals])
# print(enc.categories_)
enc_df = pd.DataFrame(enc.transform(X.loc[:, obj_vals]))
# print(X.shape, "\n", enc_df.shape)
X.reset_index(drop=True, inplace=True)
enc_df.reset_index(drop=True, inplace=True)
X.drop(obj_vals, axis=1, inplace=True)
X = pd.concat([X,enc_df], axis=1, ignore_index=False)
# print(X.info())

In [167]:
# Fill in NA values
from sklearn.impute import KNNImputer
cols_w_nans = list(get_null_pc(X)['other'])
# print (cols_w_nans)
knn_imputer = KNNImputer(n_neighbors = 1)
X[cols_w_nans] = knn_imputer.fit_transform(X[cols_w_nans])
# print(X.info())

In [168]:
# Outlier detection : up sampling ? down sampling ? Drop ?
import re
non_categ_cols = []
for col in list(X.columns) :
    if re.search("^[a-zA-Z]", str(col)) != None :
        non_categ_cols.append(col)
# Drop outliers which make < outlier_threshold
outlier_threshold = 6
for col in non_categ_cols :
    Q3, Q1 = X[col].quantile(0.75), X[col].quantile(0.25)
    IQR = Q3 - Q1
    threshold = 1.5
    outlier = X[(X[col] < Q1 - threshold * IQR) | (X[col] > Q3 + threshold * IQR)]
    pc_outlier = len(outlier) *100 / len(X)
    if pc_outlier < outlier_threshold and pc_outlier > 0 :
        X = X.drop(outlier.index)
    # else :
        # print (X[col].value_counts() / X[col].count() * 100)
        # No need for up/down sampling since max is 5% share

In [169]:
# Transform data : Standardize/Normalize
from sklearn.preprocessing import StandardScaler

X[non_categ_cols] = StandardScaler().fit_transform(X[non_categ_cols])


In [170]:
# Write out data : chip_dataset_cleaned.csv
df_out = pd.concat([X, Y], axis=1)
df_out.to_csv('../chip_dataset_cleaned.csv')

In [171]:
# Write out encodings to file
dir = "../"
for i, label in enumerate(target_label) :
    filename = dir + str(label) + ".csv"
    vendor_out = open(filename, 'w')
    vendor_out.writelines(list(les[i].classes_))
    vendor_out.close()

foundry_out = open('../Vendor.csv', 'w')
for f in enc.categories_ :
    foundry_out.write(str(f))
foundry_out.close()